# Libraries

In [1]:
from Functions import *
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "plotly_dark"
pd.options.display.float_format = '{:,.3f}'.format
from typing import Tuple

# Setup

In [2]:
stocks = ["HIMS", "ELF", "PG", "HNST"]
companies_facts = {}
tickers = {}
hisotrical_prices = {}
for stock in stocks:
    cik = get_cik_from_symbol(stock)
    companies_facts["stock"] = get_company_facts(cik, stock)
    tickers[stock] = yf.Ticker(stock)
    hisotrical_prices[stock] = tickers[stock].history(period="max", interval='1d', actions=False)

start_date = "2020-01-01"
data = yf.download([*stocks, "^GSPC"], start=start_date)["Close"]

[*********************100%***********************]  5 of 5 completed


# Overview

In [3]:
df = pd.DataFrame(stocks, columns=["symbol"])
df["shares"] = [1] * len(df)
df, _ = read_portfolio(df)
df

,symbol,shares,sector,industry,current,investment_value,price_target_low,price_target_high,price_target_mean,price_target_median,strongBuy,buy,hold,sell,strongSell
0,HIMS,1,Consumer Defensive,Household & Personal Products,26.200,26.200,18.000,38.000,24.904,24.000,2,2,7,0,1
1,ELF,1,Consumer Defensive,Household & Personal Products,128.660,128.660,115.000,200.000,157.188,156.000,3,11,5,0,0
2,PG,1,Consumer Defensive,Household & Personal Products,168.060,168.060,143.000,209.000,181.074,184.500,4,13,10,0,1
3,HNST,1,Consumer Defensive,Household & Personal Products,6.890,6.890,5.500,9.500,7.143,7.000,1,4,3,0,0


In [4]:
normalized_data = pd.DataFrame()
for column in data.columns:
    normalized_data[column] = data[column] / data[column].loc[data[column].first_valid_index()] * 100

normalized_data = normalized_data.reset_index()
melted_data = normalized_data.melt(id_vars=["Date"], var_name="Stock", value_name="Price")

fig = px.line(
    melted_data,
    x="Date",
    y="Price",
    color="Stock",
    title="Stock Prices Over Time (Normalized to 100)",
    labels={"Price": "Price (Normalized to 100)", "Date": "Date"},
)

fig.update_layout(
    title_font_size=18,
    xaxis_title="Date",
    yaxis_title="Normalized Price",
    legend_title="Stock",
)

fig.show()

# Returns

In [5]:
return_metrics = ['profitMargins', 'operatingMargins', 'returnOnAssets', 'returnOnEquity']
return_data = get_metrics(tickers, return_metrics)
return_data

,symbol,profitMargins,operatingMargins,returnOnAssets,returnOnEquity
0,HIMS,0.082,0.056,0.057,0.263
1,ELF,0.089,0.093,0.081,0.174
2,PG,0.171,0.278,0.106,0.288
3,HNST,-0.011,0.001,-0.013,-0.033


# Balance sheets 

In [6]:
balance_metrics = ['totalCash', 'totalCashPerShare', 'totalDebt', 'debtToEquity', 
                  'currentRatio', 'bookValue', ]
balance_data = get_metrics(tickers, balance_metrics)
assets = []

for _, row in balance_data.iterrows():
    if row["symbol"] == "ASML":
        assets.append(get_recent_fact_from_symbol(row["symbol"], "Assets", annual_form="20-F"))
        continue
    assets.append(get_recent_fact_from_symbol(row["symbol"], "Assets"))

balance_data["Assets"] = assets
balance_data

,symbol,totalCash,totalCashPerShare,totalDebt,debtToEquity,currentRatio,bookValue,Assets
0,HIMS,254071008,1.163,11358000,2.581,2.138,2.020,441186000
1,ELF,96768000,1.718,301265984,41.400,1.782,12.918,1129247000
2,PG,12156000256,5.162,36152999936,69.338,0.754,21.676,122370000000
3,HNST,53441000,0.529,23791000,17.772,2.890,1.323,201621000


# Income Statement

In [7]:
df = get_indicators(tickers, "income_stmt",["symbol", "Total Revenue", "Operating Income", 
                               "Net Income", "Diluted EPS", "EBITDA", 
                               "Gross Profit", "Operating Revenue"])
df

,symbol,EBITDA,Diluted EPS,Net Income,Operating Income,Gross Profit,Total Revenue,Operating Revenue
0,HIMS,"-19,938,000.000",-0.110,"-23,546,000.000","-29,453,000.000","714,949,000.000","872,000,000.000","872,000,000.000"
1,ELF,"188,313,000.000",2.210,"127,663,000.000","149,678,000.000","724,096,000.000","1,023,932,000.000","1,023,932,000.000"
2,PG,"22,582,000,000.000",6.020,"14,879,000,000.000","19,886,000,000.000","43,191,000,000.000","84,039,000,000.000","84,039,000,000.000"
3,HNST,"-36,154,000.000",-0.420,"-39,238,000.000","-36,704,000.000","100,532,000.000","344,365,000.000","344,365,000.000"


# Cash Flow Statement

In [8]:
df = get_indicators(tickers, "cash_flow",["symbol", "year", "Free Cash Flow", "Operating Cash Flow", 
                             "Cash Flow From Continuing Investing Activities", "Cash Flow From Continuing Financing Activities",
                             "Cash Flow From Continuing Operating Activities"])
df = df.fillna(0)

df

,symbol,year,Free Cash Flow,Cash Flow From Continuing Financing Activities,Cash Flow From Continuing Investing Activities,Operating Cash Flow,Cash Flow From Continuing Operating Activities
0,HIMS,2023-12-31,"46,991,000.000","-11,475,000.000","-12,106,000.000","73,483,000.000","73,483,000.000"
1,ELF,2024-03-31,"62,495,000.000","200,945,000.000","-284,660,000.000","71,154,000.000","71,154,000.000"
2,PG,2024-06-30,"16,524,000,000.000","-14,855,000,000.000","-3,504,000,000.000","19,846,000,000.000","19,847,000,000.000"
3,HNST,2023-12-31,"17,505,000.000","122,000.000","3,835,000.000","19,353,000.000","19,353,000.000"
